In [ ]:
#import os
import numpy.ma as ma
from netCDF4 import Dataset, date2index, num2date
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import xarray as xr
#from math import atan2, log
from palettable import colorbrewer
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlaba
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)
from bs4 import BeautifulSoup
import requests
def listFD(url, ext=''):
    page = requests.get(url).text
    #print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') if (node.get('href').endswith(ext) and node.get('href').startswith('2'))]
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt


In [ ]:
filename_in='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1.nc'
#dataset=xr.open_dataset(temp_out,decode_times=False)
dataset=xr.open_dataset(filename_in)

In [ ]:
dataset

In [ ]:
dataset.TIME.min()

In [ ]:
istart_data=1
xtime=pd.to_datetime(dataset.TIME.data)
xdyofyear=xtime.timetuple().tm_yday
minday=dataset.TIME.min()
maxday=dataset.TIME.max()
x = pd.to_datetime(minday.data)
minday=x.timetuple().tm_yday
x = pd.to_datetime(maxday.data)
maxday=x.timetuple().tm_yday
print(minday,maxday)
print(dt.datetime(2018, 1, 1) + dt.timedelta(minday - 1))

In [ ]:
istart_data=1
for days in range(minday,maxday+1):
    inew_data=1
    for incr_day in range(-1,2):
        d = dt.datetime(2018, 1, 1) + dt.timedelta(days - 1) + timedelta(days=incr_day)
        day_of_year = d.timetuple().tm_yday
        url = 'https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L3U/VIIRS_NPP/OSPO/v2.41/' \
            + str(d.year) + '/' + str(day_of_year).zfill(3) + '/'
        ext = 'nc'
        
        filenames=listFD(url, ext)
        ilen=len(filenames)
        for ic in range(1,ilen):
            file = filenames[ic]
            print(file)
            nc = Dataset(file)
            sst = nc.variables['sea_surface_temperature'][:,2500:3250,2500:3500]
            sst = np.flip(sst, 2)
            cnt = ma.count(sst)
            lat = nc.variables['lat'][2500:3250]
            lat = np.flip(lat, 0)
            lon = nc.variables['lon'][2500:3500]
            nc.close()
            if istart_data==1:
                sst_new_all = ma.copy(sst)
                istart_data = 0
            if inew_data==1:
                sst_new = ma.copy(sst)
                inew_data = 0
            if cnt<5:
                continue
            print(ic,cnt,file)
            mask = sst_new < -10
            sst_new[mask] = sst[mask]
            sst_new_all[mask] = sst[mask]

    sst_new2 = sst_new[0,:,:]
    sstx = ma.masked_values (sst_new2, -9999.)
    sst3x = np.flip(sstx, 1)
    sst4x = np.flip(sst3x, 0)
    sst5x = ma.swapaxes(sst4x,0,1)
    
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
    fig = plt.figure(figsize=(11.7,8.3))
    m = Basemap(projection='merc', lat_0 = 27, lon_0 = -120,    resolution = 'l', area_thresh = 0.1,    
                              llcrnrlon=-126.0, llcrnrlat=25.0,urcrnrlon=-114.0, urcrnrlat=38.)
    m.bluemarble()
#    m.fillcontinents(color='grey',lake_color='white')
    lat_grid, lon_grid = np.meshgrid(lat, lon)
    x,y = m(lon_grid,lat_grid)
    cs = m.pcolormesh(x,y, sst5x - 273.15, cmap=cmap, vmin = 12, vmax = 19)
    sub_lons=lons_usv[days==day_of_year]
    sub_lats=lats_usv[days==day_of_year]
#    sub_lons=lons_usv[tem_dyr==day_of_year]
#    sub_lats=lats_usv[tem_dyr==day_of_year]
    x,y = m(sub_lons,sub_lats)
    m.plot(x,y,'w', linewidth=3)
    m.plot(x,y,color='magenta', linewidth=1)
    m.colorbar(location="bottom",label='SST [C]') # draw colorbar
    plt.show()
    filename_png='F:/data/cruise_data/saildrone/baja-2018/figs/cruise_sst_'+str(day_of_year)+'.png'
    fig.savefig(filename_png, transparent=False, format='png')
    print('done') 
    
    sub_lons=lons_usv[tem_dyr==day_of_year]
    sub_lats=lats_usv[tem_dyr==day_of_year]
    min_lon=min(sub_lons)-3
    max_lon=min(sub_lons)+3
    min_lat=min(sub_lats)-3
    max_lat=min(sub_lats)+3
    center_lon=.5*(max_lon-min_lon)+min_lon
    center_lat=.5*(max_lat-min_lat)+min_lat
    print(min_lon,max_lon,min_lat,max_lat,center_lon,center_lat)
    
    fig = plt.figure(figsize=(11.7,8.3))
    m = Basemap(projection='merc', lat_0 = center_lat, lon_0 = center_lon,    resolution = 'l', area_thresh = 0.1,    
                llcrnrlon=min_lon, llcrnrlat=min_lat,urcrnrlon=max_lon, urcrnrlat=max_lat)
    m.bluemarble()
#    m.fillcontinents(color='grey',lake_color='white')
    lat_grid, lon_grid = np.meshgrid(lat, lon)
    x,y = m(lon_grid,lat_grid)
    cs = m.pcolormesh(x,y, sst5x - 273.15, cmap=cmap, vmin = 12, vmax = 19)
    x,y = m(sub_lons,sub_lats)
    m.plot(x,y,'w', linewidth=3)
    m.plot(x,y,color='magenta', linewidth=1)
    m.colorbar(location="bottom",label='SST [C]') # draw colorbar
    plt.show()
    filename_png='F:/data/cruise_data/saildrone/baja-2018/figs/cruise_sst_'+str(day_of_year)+'_zoom.png'
    fig.savefig(filename_png, transparent=False, format='png')
    print('done')      

sst_new2 = sst_new_all[0,:,:]
sstx = ma.masked_values (sst_new2, -9999.)
sst3x = np.flip(sstx, 1)
sst4x = np.flip(sst3x, 0)
sst5x = ma.swapaxes(sst4x,0,1)
sub_lons=lons_usv
sub_lats=lats_usv
fig = plt.figure(figsize=(11.7,8.3))
m = Basemap(projection='merc', lat_0 = 27, lon_0 = -120,    resolution = 'l', area_thresh = 0.1,    
                              llcrnrlon=-126.0, llcrnrlat=25.0,urcrnrlon=-114.0, urcrnrlat=38.)
m.bluemarble()
    #m.fillcontinents(color='grey',lake_color='white')
lat_grid, lon_grid = np.meshgrid(lat, lon)
x,y = m(lon_grid,lat_grid)
cs = m.pcolormesh(x,y, sst5x - 273.15, cmap=cmap, vmin = 12, vmax = 19)
x,y = m(sub_lons,sub_lats)
m.plot(x,y,'w', linewidth=3)
m.plot(x,y,color='magenta', linewidth=1)
m.colorbar(location="bottom",label='SST [C]') # draw colorbar
plt.show()
filename_png='F:/data/cruise_data/saildrone/baja-2018/figs/cruise_all_sst_track.png'
fig.savefig(filename_png, transparent=False, format='png')
print('done')   


In [ ]:
cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
fig = plt.figure(figsize=(11.7,8.3))
m = Basemap(projection='merc', lat_0 = 27, lon_0 = -120,    resolution = 'l', area_thresh = 0.1,    
                              llcrnrlon=-126.0, llcrnrlat=25.0,urcrnrlon=-114.0, urcrnrlat=38.)
m.bluemarble()
#    m.fillcontinents(color='grey',lake_color='white')
lat_grid, lon_grid = np.meshgrid(lat, lon)
x,y = m(lon_grid,lat_grid)
cs = m.pcolormesh(x,y, sst5x - 273.15, cmap=cmap, vmin = 12, vmax = 19)
sub_lons=dataset.LON.data[days==day_of_year]
sub_lats=dataset.LAT.data[days==day_of_year]
print(sub_lons)
#    sub_lons=lons_usv[tem_dyr==day_of_year]
#    sub_lats=lats_usv[tem_dyr==day_of_year]
x,y = m(sub_lons,sub_lats)
m.plot(x,y,'w', linewidth=3)
m.plot(x,y,color='magenta', linewidth=1)
m.colorbar(location="bottom",label='SST [C]') # draw colorbar
plt.show()
filename_png='F:/data/cruise_data/saildrone/baja-2018/figs/cruise_sst_'+str(day_of_year)+'.png'
fig.savefig(filename_png, transparent=False, format='png')
print('done') 


In [ ]:
#dataset.loc[dict(time=slice('2000-01-01', '2000-01-02'))]
dataset
#dataset.index=dataset['TIME']

#dataset.sel(dataset.TIME=dt.datetime(2018, 1, 1) + dt.timedelta(days - 1))
#dataset.where(dataset.TIME.data='2000-01-01')

In [ ]:
xtime=pd.to_datetime(dataset.TIME.data)
#print(xtime)
#xdyofyear=xtime.timetuple().tm_yday
d = dt.datetime(2018, 1, 1) + dt.timedelta(days - 1) 
print(d)
#print(dataset.TIME)
#xtime=dataset.TIME[0,:]
#print(xtime)
dataset.LAT[xtime==d]
dataset.TIME